In [12]:
!pip install opendatasets as od
import opendatasets as od
od.download("https://www.kaggle.com/datasets/senapatirajesh/netflix-tv-shows-and-movies")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:

In [1]:
import pandas as pd
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
import dotenv

In [3]:
data_df = pd.read_csv("netflix_titles.csv")[["title","description"]]
print(data_df.isna().sum())
print()
print(data_df.shape)

title          0
description    0
dtype: int64

(8807, 2)


In [4]:
Embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                            model_kwargs={'device': 'mps'})

In [5]:
REVIEWS_CSV_PATH = "data/reviews.csv"
REVIEWS_CHROMA_PATH = "chroma_data"

dotenv.load_dotenv()

loader = DataFrameLoader(data_df, page_content_column="description")
descriptions = loader.load()

reviews_vector_db = Chroma.from_documents(
    descriptions, Embeddings, persist_directory=REVIEWS_CHROMA_PATH
)

OperationalError: no such column: collections.topic

In [137]:
question = """
movie talking about romantic love story between couple fighting to live together
"""

relevant_docs = reviews_vector_db.similarity_search(question, k=3)

In [138]:
print("Most Suggested Movie:")
print()
title = relevant_docs[0].metadata["title"]
print("Movie Name: ", data_df[data_df["title"] == title]["title"].iloc[0])
print()
print("Movie Description: ", data_df[data_df["title"] == title]["description"].iloc[0])

print()
print()

print("Least Suggested Movie:")
print()
title = relevant_docs[-1].metadata["title"]
print("Movie Name: ", data_df[data_df["title"] == title]["title"].iloc[0])
print()
print("Movie Description: ", data_df[data_df["title"] == title]["description"].iloc[0])

Most Suggested Movie:

Movie Name:  All Good Ones Get Away

Movie Description:  When a mysterious figure blackmails an adulterous couple during a romantic getaway, their secret affair turns into a fight for survival.


Least Suggested Movie:

Movie Name:  All About Love

Movie Description:  An inseparable couple struggles to stay together when betrayal threatens to tear their lives apart.


In [ ]:
reviews_vector_db = Chroma(persist_directory=REVIEWS_CHROMA_PATH, embedding_function=Embeddings)

In [1]:
from movies import movie_rag
import pandas as pd
df = pd.read_csv("netflix_titles.csv")

movie_obj = movie_rag(df=df,REVIEWS_CHROMA_PATH = 'chroma_data', device='mps')

/Users/omaratef/miniconda/envs/pytorch/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/omaratef/miniconda/envs/pytorch/lib/python3.9/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/omaratef/miniconda/envs/pytorch/lib/python3.9/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads alway

In [2]:
movie_obj.create_vector_database()

In [3]:
vector_db = movie_obj.load_vector_database()

movies = movie_obj.get_movie("romantic love story between couple fighting to live together",  vector_db)

/Users/omaratef/miniconda/envs/pytorch/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [4]:
print(movies[0].metadata["title"])
print()
print(movies[0].page_content)

Irada Pakka

Bored in their happy marriage, a couple decides to spice things up by fighting more, but their game takes the relationship on an unexpected path.


In [5]:
print(movies[1].metadata["title"])
print()
print(movies[1].page_content)

All About Love

An inseparable couple struggles to stay together when betrayal threatens to tear their lives apart.
